In [1]:
%%bash

dx download "/genetics_of_miscarriage/command_files/pheno_extraction/extract_phenotypes_miscarriage_genetics.ipynb"

### 1. Import pkgs and init

In [2]:
import pyspark
import dxpy
import dxdata
import pandas as pd

In [3]:
# init spark
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

In [4]:
# Automatically discover dispensed dataset name and id - specific to each project

dispensed_database_name = dxpy.find_one_data_object(classname = "database",
                                                   name = "app*", folder = "/",
                                                   name_mode = "glob",
                                                   describe = True) ["describe"] ["name"]

dispensed_dataset_id = dxpy.find_one_data_object(typename = "Dataset",
                                                name = "app*.dataset", folder = "/",
                                                name_mode = "glob") ["id"]



### 2. Load datatset and def entity

In [5]:
dataset = dxdata.load_dataset(id = dispensed_dataset_id)

In [6]:
dataset.entities

[<Entity "participant">,
 <Entity "covid19_result_england">,
 <Entity "covid19_result_scotland">,
 <Entity "covid19_result_wales">,
 <Entity "death">,
 <Entity "death_cause">,
 <Entity "gp_clinical">,
 <Entity "gp_registrations">,
 <Entity "gp_scripts">,
 <Entity "hesin">,
 <Entity "hesin_critical">,
 <Entity "hesin_delivery">,
 <Entity "hesin_diag">,
 <Entity "hesin_maternity">,
 <Entity "hesin_oper">,
 <Entity "hesin_psych">]

In [7]:
participant = dataset["participant"]

### 3. Import relevant fields

In [8]:
%%bash

dx download "/genetics_of_miscarriage/original_data/fem_spec_facs_ukb_field_names_pyspark.txt"

In [9]:
with open("fem_spec_facs_ukb_field_names_pyspark.txt") as f:
    field_names = [line.rstrip('\n') for line in f]

del field_names[0]
field_names.extend(["eid"])


In [14]:
field_names.remove("p10132") # note that this works only on 1st instance of element with this name

In [20]:
field_names.extend(["p31"])

In [15]:
print(field_names)

['p2674_i0', 'p2674_i1', 'p2674_i2', 'p2674_i3', 'p2684_i0', 'p2684_i1', 'p2684_i2', 'p2684_i3', 'p2694_i0', 'p2694_i1', 'p2694_i2', 'p2694_i3', 'p2704_i0', 'p2704_i1', 'p2704_i2', 'p2704_i3', 'p2714_i0', 'p2714_i1', 'p2714_i2', 'p2714_i3', 'p2724_i0', 'p2724_i1', 'p2724_i2', 'p2724_i3', 'p2734_i0', 'p2734_i1', 'p2734_i2', 'p2734_i3', 'p2744_i0', 'p2744_i1', 'p2744_i2', 'p2744_i3', 'p2754_i0', 'p2754_i1', 'p2754_i2', 'p2754_i3', 'p2764_i0', 'p2764_i1', 'p2764_i2', 'p2764_i3', 'p2774_i0', 'p2774_i1', 'p2774_i2', 'p2774_i3', 'p2784_i0', 'p2784_i1', 'p2784_i2', 'p2784_i3', 'p2794_i0', 'p2794_i1', 'p2794_i2', 'p2794_i3', 'p2804_i0', 'p2804_i1', 'p2804_i2', 'p2804_i3', 'p2814_i0', 'p2814_i1', 'p2814_i2', 'p2814_i3', 'p2824_i0', 'p2824_i1', 'p2824_i2', 'p2824_i3', 'p2834_i0', 'p2834_i1', 'p2834_i2', 'p2834_i3', 'p3536_i0', 'p3536_i1', 'p3536_i2', 'p3536_i3', 'p3546_i0', 'p3546_i1', 'p3546_i2', 'p3546_i3', 'p3581_i0', 'p3581_i1', 'p3581_i2', 'p3581_i3', 'p3591_i0', 'p3591_i1', 'p3591_i2', 'p3

In [21]:
df = participant.retrieve_fields(names = field_names,
                                #filter_sql = cohort.sql
                                coding_values = "replace", #Replaces coded values with actual values e.g. 1/0 with M/F
                                engine = dxdata.connect())

[dxdata.dataset.dataset] WARNING: Dataframe schema name(s): ['participant_0006$p2674_i0', 'participant_0006$p2674_i1', 'participant_0006$p2674_i2', 'participant_0006$p2674_i3', 'participant_0006$p2684_i0', 'participant_0006$p2684_i1', 'participant_0006$p2684_i2', 'participant_0006$p2684_i3', 'participant_0006$p2694_i0', 'participant_0006$p2694_i1', 'participant_0006$p2694_i2', 'participant_0006$p2694_i3', 'participant_0006$p2704_i0', 'participant_0006$p2704_i1', 'participant_0006$p2704_i2', 'participant_0006$p2704_i3', 'participant_0006$p2714_i0', 'participant_0006$p2714_i1', 'participant_0006$p2714_i2', 'participant_0006$p2714_i3', 'participant_0006$p2724_i0', 'participant_0006$p2724_i1', 'participant_0006$p2724_i2', 'participant_0006$p2724_i3', 'participant_0006$p2734_i0', 'participant_0006$p2734_i1', 'participant_0006$p2734_i2', 'participant_0006$p2734_i3', 'participant_0006$p2744_i0', 'participant_0006$p2744_i1', 'participant_0006$p2744_i2', 'participant_0006$p2744_i3', 'participan

In [22]:
df.limit(5).toPandas()

,p2674_i0,p2674_i1,p2674_i2,p2674_i3,p2684_i0,p2684_i1,p2684_i2,p2684_i3,p2694_i0,p2694_i1,...,p3872_i0,p3872_i1,p3872_i2,p3872_i3,p3882_i0,p3882_i1,p3882_i2,p3882_i3,eid,p31
0,Yes,None,None,None,3,None,None,None,Yes,None,...,27,None,None,None,None,None,None,None,1000020,Female
1,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,1000104,Male
2,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,1000181,Male
3,No,None,None,None,None,None,None,None,Yes,None,...,None,None,None,None,None,None,None,None,1000254,Female
4,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,1000268,Male


### 4. Field matching

In [23]:
def fields_for_id(field_id):
    from distutils.version import LooseVersion
    field_id = str(field_id)
    fields = participant.find_fields(name_regex='^p{}(_i\d+)?(_a\d+)?$'.format(field_id))
    return sorted(fields, key=lambda f: LooseVersion(f.name))

In [24]:
def field_names_for_id(field_id):
    return[f.name for f in fields_for_id(field_id)]

### 5. Extract relevant data for females

In [25]:
print(df.count())

502415


In [26]:
df_female = df.filter("p31 = 'Female'")

In [27]:
print(df_female.count())

273329


In [28]:
df_female.toPandas().to_csv("miscarriage_genetics_female_specific_phenos.tsv", sep="\t", index=False)

In [29]:
%%bash
dx upload miscarriage_genetics_female_specific_phenos.tsv --dest /genetics_of_miscarriage/original_data/miscarriage_genetics_female_specific_phenos.tsv

ID                    file-G6yyyyjJZGX7gv6F1z9KBv29
Class                 file
Project               project-G6Z05V8JZGX3JBB9JpBqX6V7
Folder                /genetics_of_miscarriage/original_data
Name                  miscarriage_genetics_female_specific_phenos.tsv
State                 closing
Visibility            visible
Types                 -
Properties            -
Tags                  -
Outgoing links        -
Created               Sat Dec 18 14:26:03 2021
Created by            alexander_moerseburg
 via the job          job-G6yyYFjJZGX6BP9J6zXQ244k
Last modified         Sat Dec 18 14:26:06 2021
Media type            
archivalState         "live"
cloudAccount          "cloudaccount-dnanexus"
